In [1]:
#codebase
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data,HeteroData
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GATConv,HeteroConv
from torch_geometric.nn import global_mean_pool
from sklearn.metrics import ndcg_score
from sklearn.model_selection import train_test_split


In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def create_graph(df):
    # 对 userId 和 movieId 重新编码为从 0 开始的连续整数
    df = df.copy()
    df['userId'] = df['userId'].astype('category').cat.codes
    df['movieId'] = df['movieId'].astype('category').cat.codes

    num_users = df['userId'].nunique()
    num_movies = df['movieId'].nunique()
    graph = HeteroData()
    
    # 确保 user.x 和 movie.x 的维度足够大
    graph['user'].x = torch.arange(num_users, dtype=torch.float).view(-1, 1)

    merged_df = pd.merge(df,movies,how='left',on='movieId')
    merged_df = merged_df[['movieId','genres']].drop_duplicates()
    genres = merged_df['genres'].str.get_dummies(sep='|')
    item_features = torch.tensor(genres.values, dtype=torch.float)
    graph['movie'].x = item_features
    
    # 检查 edge_index 的索引范围
    edge_index = torch.tensor([df['userId'].values, df['movieId'].values], dtype=torch.long)
    
    assert edge_index[0].max() < num_users, "User ID 越界"
    assert edge_index[1].max() < num_movies, "Movie ID 越界"
    
    graph['user', 'rates', 'movie'].edge_index = edge_index
    graph['user', 'rates', 'movie'].edge_attr = torch.tensor(df['rating'].values, dtype=torch.float) 
    graph['movie', 'rated_by', 'user'].edge_index = edge_index[[1, 0]]  # 更安全的翻转方式
    
    return graph

In [4]:
# 划分训练集和测试集
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

In [5]:
train_graph = create_graph(train_ratings).to(device)
test_graph = create_graph(test_ratings).to(device)

C:\Users\78529\AppData\Local\Temp\ipykernel_19512\1530743362.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  edge_index = torch.tensor([df['userId'].values, df['movieId'].values], dtype=torch.long)


In [6]:
train_graph

HeteroData(
  user={ x=[200948, 1] },
  movie={ x=[80318, 19] },
  (user, rates, movie)={
    edge_index=[2, 25600163],
    edge_attr=[25600163],
  },
  (movie, rated_by, user)={ edge_index=[2, 25600163] }
)

In [7]:
print("用户节点数:", train_graph['user'].x.shape[0])
print("电影节点数:", train_graph['movie'].x.shape[0])
print("边索引最大值 - user:", train_graph['user', 'rates', 'movie'].edge_index[0].max().item())
print("边索引最大值 - movie:", train_graph['user', 'rates', 'movie'].edge_index[1].max().item())

用户节点数: 200948
电影节点数: 80318
边索引最大值 - user: 200947
边索引最大值 - movie: 80317


In [41]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.lin_dict = torch.nn.ModuleDict({
            'user': torch.nn.LazyLinear(hidden_channels),
            'movie': torch.nn.LazyLinear(hidden_channels)
        })
        
        self.norm_dict = torch.nn.ModuleDict({
            'user': torch.nn.BatchNorm1d(hidden_channels),
            'movie': torch.nn.BatchNorm1d(hidden_channels)
        })
        
        self.dropout = torch.nn.Dropout(0.2)
        
        self.conv1 = HeteroConv({
            ('user', 'rates', 'movie'): SAGEConv((-1, -1), hidden_channels),
            ('movie', 'rated_by', 'user'): SAGEConv((-1, -1), hidden_channels),
        }, aggr='mean')
        
        self.conv2 = HeteroConv({
            ('user', 'rates', 'movie'): SAGEConv((-1, -1), out_channels),
            ('movie', 'rated_by', 'user'): SAGEConv((-1, -1), out_channels),
        }, aggr='mean')
    
    def forward(self, x_dict, edge_index_dict):
        # 特征变换
        x_dict = {key: self.lin_dict[key](x) for key, x in x_dict.items()}
        x_dict = {key: self.norm_dict[key](x) for key, x in x_dict.items()}
        x_dict = {key: self.dropout(F.relu(x)) for key, x in x_dict.items()}
        
        # 图卷积
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: self.dropout(F.relu(x)) for key, x in x_dict.items()}
        x_dict = self.conv2(x_dict, edge_index_dict)
        
        return x_dict

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, 4 * hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(4 * hidden_channels, 2 * hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(2 * hidden_channels, 1)
        )
    
    def forward(self, z_dict, edge_label_index):
        user_z = z_dict['user'][edge_label_index[0]]
        movie_z = z_dict['movie'][edge_label_index[1]]
        features = torch.cat([user_z, movie_z], dim=-1)
        return self.mlp(features).squeeze(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, metadata):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.decoder = EdgeDecoder(hidden_channels)
    
    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [42]:
user_feat_dim = train_graph['user'].x.size(1)
movie_feat_dim = train_graph['movie'].x.size(1)

model = Model(hidden_channels=64, metadata=train_graph.metadata()).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
criterion = torch.nn.MSELoss()

In [43]:
from torch_geometric.loader import LinkNeighborLoader

def prepare_loaders(train_graph, test_graph, batch_size=512):
    # 训练加载器 - 包含评分作为边属性
    train_loader = LinkNeighborLoader(
        data=train_graph,
        num_neighbors=[20, 10],  # 采样邻居数
        edge_label_index=(('user', 'rates', 'movie'), train_graph[('user', 'rates', 'movie')].edge_index),
        edge_label=train_graph[('user', 'rates', 'movie')].edge_attr,  # 使用实际评分
        batch_size=batch_size,
        shuffle=True,
    )
    
    # 测试加载器
    test_loader = LinkNeighborLoader(
        data=test_graph,
        num_neighbors=[20, 10],
        edge_label_index=(('user', 'rates', 'movie'), test_graph[('user', 'rates', 'movie')].edge_index),
        edge_label=test_graph[('user', 'rates', 'movie')].edge_attr,
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_loader, test_loader

In [44]:
train_loader, test_loader = prepare_loaders(train_graph, test_graph, batch_size=4096)


In [46]:
def train(model, loader, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        
        pred = model(batch.x_dict, batch.edge_index_dict, 
                    batch[('user', 'rates', 'movie')].edge_label_index)
        
        # 使用均方误差损失
        loss = F.mse_loss(pred, batch[('user', 'rates', 'movie')].edge_label)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.size(0)
    
    return total_loss / len(loader.dataset)

def evaluate(model, loader, device):
    model.eval()
    total_mse = 0
    total_mae = 0
    
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pred = model(batch.x_dict, batch.edge_index_dict,
                        batch[('user', 'rates', 'movie')].edge_label_index)
            
            mse = F.mse_loss(pred, batch[('user', 'rates', 'movie')].edge_label)
            mae = F.l1_loss(pred, batch[('user', 'rates', 'movie')].edge_label)
            
            total_mse += float(mse) * pred.size(0)
            total_mae += float(mae) * pred.size(0)
    
    return {
        'mse': total_mse / len(loader.dataset),
        'mae': total_mae / len(loader.dataset),
        'rmse': np.sqrt(total_mse / len(loader.dataset))
    }

In [37]:
train_graph[('user', 'rates', 'movie')]

{'edge_index': tensor([[161934,  23151, 152930,  ..., 100886, 145265, 198409],
        [  2233,  28513,   4252,  ...,  64923,  10397,   5241]],
       device='cuda:0'), 'edge_attr': tensor([5.0000, 5.0000, 4.5000,  ..., 3.5000, 3.0000, 2.5000], device='cuda:0')}

In [47]:
def generate_recommendations(model, data, user_id, top_k=10, device='cpu'):
    """
    为指定用户生成电影推荐
    
    参数:
        model: 训练好的推荐模型
        data: 包含图数据的HeteroData对象
        user_id: 要推荐的用户ID(原始ID或编码后ID)
        top_k: 返回的推荐数量
        device: 计算设备('cpu'或'cuda')
    
    返回:
        tuple: (推荐电影ID列表, 预测评分列表)
    """
    model.eval()
    data = data.to(device)
    
    with torch.no_grad():
        # 获取所有节点表示
        z_dict = model.encoder(data.x_dict, data.edge_index_dict)
        
        # 确保user_id是tensor格式
        if isinstance(user_id, int):
            user_id = torch.tensor([user_id], device=device)
        
        # 获取目标用户表示
        user_emb = z_dict['user'][user_id]  # shape: [1, emb_dim]
        movie_emb = z_dict['movie']         # shape: [num_movies, emb_dim]
        
        # 计算用户对所有电影的预测分数 (使用解码器)
        edge_label_index = torch.stack([
            user_id.repeat(movie_emb.size(0)),
            torch.arange(movie_emb.size(0), device=device)
        ])
        scores = model.decoder(z_dict, edge_label_index)  # shape: [num_movies]
        
        # 排除已评分的电影
        rated_mask = torch.zeros(movie_emb.size(0), dtype=torch.bool, device=device)
        user_edges = (data['user', 'rates', 'movie'].edge_index[0] == user_id)
        rated_movies = data['user', 'rates', 'movie'].edge_index[1][user_edges]
        rated_mask[rated_movies] = True
        scores[rated_mask] = -float('inf')
        
        # 获取top-k推荐及其分数
        top_scores, top_movies = torch.topk(scores, k=min(top_k, len(scores)))
        
        return top_movies.cpu().numpy(), top_scores.cpu().numpy()

In [48]:
# 训练循环
print("开始训练...")
# 训练循环
for epoch in range(1, 6):
    train_loss = train(model, train_loader, optimizer, device)
    test_metrics = evaluate(model, test_loader, device)
    
    scheduler.step(epoch)
    
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
          f'Test RMSE: {test_metrics["rmse"]:.4f}, MAE: {test_metrics["mae"]:.4f}')
    
# 保存模型
torch.save(model.state_dict(), 'movie_recommender.pth')



开始训练...


KeyboardInterrupt: 

In [ ]:
# 示例推荐
user_id = 0  # 选择第一个用户
recommended_movies = generate_recommendations(model, train_graph, user_id)
movie_titles = movies.set_index('movieId')['title'].to_dict()
print(f"\n为用户 {user_id} 推荐的电影:")
for i, movie_id in enumerate(recommended_movies, 1):
    original_id = train_graph['movieId'].unique()[movie_id]
    print(f"{i}. {movie_titles.get(original_id, '未知电影')}")